In [1]:
import pandas as pd
import numpy as np


In [2]:
import tensorflow
tensorflow.keras.__version__

'2.4.0'

In [3]:
!pip install sklearn --upgrade

In [4]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [5]:
data = "../Resources/modelling.csv"
modelling = pd.read_csv(data)
modelling

,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
0,A2,Y,51,F,501,189.0,Physical,Medium,RTW,2021-02-25,2020-11-06
1,C2,Y,60,M,721,42.0,Physical,Small,RTW,2021-03-15,2020-11-05
2,C2,Y,29,F,459,156.0,Physical,Small,RTW,2021-12-02,2020-11-01
3,DH,N,19,M,224,4.0,Physical,Small,No RTW,2021-11-09,2020-11-02
4,C2,N,45,F,459,154.0,Physical,Medium,No RTW,NaN,2020-11-02
...,...,...,...,...,...,...,...,...,...,...,...
3679,AV,N,50,F,219,10.0,Physical,Small,No RTW,NaN,2021-02-26
3680,C1,Y,37,F,702,99.0,Mental,Small,RTW,2021-08-23,2021-03-02
3681,C1,N,59,F,703,403.0,Mental,Small,NaN,NaN,2021-07-05
3682,B1,Y,25,M,203,6.0,Physical,Small,NaN,2021-09-27,2021-07-13


In [6]:
model_df = modelling.dropna(subset=['Status26plus'])

In [7]:
le1 = LabelEncoder()
le2 = LabelEncoder()
le3 = LabelEncoder()
model_df.Gender = le1.fit_transform(model_df.Gender)
model_df.Physical_or_Mental_Injury = le2.fit_transform(model_df.Physical_or_Mental_Injury)
model_df.Employer_Type = le3.fit_transform(model_df.Employer_Type)
model_df

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
0,A2,Y,51,0,501,189.0,1,3,RTW,2021-02-25,2020-11-06
1,C2,Y,60,1,721,42.0,1,4,RTW,2021-03-15,2020-11-05
2,C2,Y,29,0,459,156.0,1,4,RTW,2021-12-02,2020-11-01
3,DH,N,19,1,224,4.0,1,4,No RTW,2021-11-09,2020-11-02
4,C2,N,45,0,459,154.0,1,3,No RTW,NaN,2020-11-02
...,...,...,...,...,...,...,...,...,...,...,...
3675,B1,Y,55,1,721,5647.0,1,3,RTW,2021-06-14,2020-12-18
3676,B1,N,56,1,222,133.0,1,3,No RTW,2021-10-11,2021-04-08
3677,C2,Y,24,0,459,16.0,1,3,RTW,2021-07-28,2021-05-04
3679,AV,N,50,0,219,10.0,1,4,No RTW,NaN,2021-02-26


In [8]:
X = model_df.drop(["Working", "Status26plus", "ARD", 'RTW_Date', "Team"], axis=1)
y = model_df["Working"]
#print(X.shape, y.shape)
X.shape

(2398, 6)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=1)
X_train.shape

(1798, 6)

In [10]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [12]:
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

In [13]:
# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='relu', input_dim=6))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=2, activation='sigmoid'))

In [14]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               700       
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 21,102
Trainable params: 21,102
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=200,
    shuffle=True,
    verbose=2
)

Epoch 1/200
57/57 - 0s - loss: 0.6269 - accuracy: 0.6763
Epoch 2/200
57/57 - 0s - loss: 0.5906 - accuracy: 0.7030
Epoch 3/200
57/57 - 0s - loss: 0.5845 - accuracy: 0.7097
Epoch 4/200
57/57 - 0s - loss: 0.5811 - accuracy: 0.7013
Epoch 5/200
57/57 - 0s - loss: 0.5803 - accuracy: 0.7058
Epoch 6/200
57/57 - 0s - loss: 0.5781 - accuracy: 0.7047
Epoch 7/200
57/57 - 0s - loss: 0.5783 - accuracy: 0.7152
Epoch 8/200
57/57 - 0s - loss: 0.5776 - accuracy: 0.7102
Epoch 9/200
57/57 - 0s - loss: 0.5758 - accuracy: 0.7102
Epoch 10/200
57/57 - 0s - loss: 0.5755 - accuracy: 0.7080
Epoch 11/200
57/57 - 0s - loss: 0.5731 - accuracy: 0.7125
Epoch 12/200
57/57 - 0s - loss: 0.5737 - accuracy: 0.7119
Epoch 13/200
57/57 - 0s - loss: 0.5721 - accuracy: 0.7108
Epoch 14/200
57/57 - 0s - loss: 0.5726 - accuracy: 0.7152
Epoch 15/200
57/57 - 0s - loss: 0.5693 - accuracy: 0.7113
Epoch 16/200
57/57 - 0s - loss: 0.5718 - accuracy: 0.7097
Epoch 17/200
57/57 - 0s - loss: 0.5715 - accuracy: 0.7141
Epoch 18/200
57/57 - 0s

Epoch 142/200
57/57 - 0s - loss: 0.5276 - accuracy: 0.7308
Epoch 143/200
57/57 - 0s - loss: 0.5229 - accuracy: 0.7341
Epoch 144/200
57/57 - 0s - loss: 0.5267 - accuracy: 0.7258
Epoch 145/200
57/57 - 0s - loss: 0.5249 - accuracy: 0.7380
Epoch 146/200
57/57 - 0s - loss: 0.5217 - accuracy: 0.7330
Epoch 147/200
57/57 - 0s - loss: 0.5227 - accuracy: 0.7325
Epoch 148/200
57/57 - 0s - loss: 0.5231 - accuracy: 0.7369
Epoch 149/200
57/57 - 0s - loss: 0.5208 - accuracy: 0.7341
Epoch 150/200
57/57 - 0s - loss: 0.5230 - accuracy: 0.7308
Epoch 151/200
57/57 - 0s - loss: 0.5196 - accuracy: 0.7341
Epoch 152/200
57/57 - 0s - loss: 0.5257 - accuracy: 0.7319
Epoch 153/200
57/57 - 0s - loss: 0.5174 - accuracy: 0.7403
Epoch 154/200
57/57 - 0s - loss: 0.5216 - accuracy: 0.7347
Epoch 155/200
57/57 - 0s - loss: 0.5197 - accuracy: 0.7369
Epoch 156/200
57/57 - 0s - loss: 0.5200 - accuracy: 0.7358
Epoch 157/200
57/57 - 0s - loss: 0.5201 - accuracy: 0.7303
Epoch 158/200
57/57 - 0s - loss: 0.5208 - accuracy: 0.73

In [17]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

19/19 - 0s - loss: 0.6305 - accuracy: 0.7050
Normal Neural Network - Loss: 0.6305041909217834, Accuracy: 0.7049999833106995


In [18]:
encoded_predictions = model.predict_classes(X_test_scaled[:20])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [19]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:20])}")

Predicted classes: ['Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'Y' 'N' 'Y' 'Y' 'N' 'Y' 'Y'
 'Y' 'Y']
Actual Labels: ['Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'N']


In [20]:
##Model rtw outcomes
predict_df = modelling[modelling['Status26plus'].isna()]
#predict_df = predict_df[predict_df['RTW_Date'].isna()]
predict_df


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
70,AV,Y,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22
181,B2,Y,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18
1724,B2,N,41,F,542,213.0,Physical,Large,NaN,NaN,2021-06-17
2061,B1,Y,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09
2227,C1,N,53,M,542,117.0,Physical,Small,NaN,NaN,2021-07-01
...,...,...,...,...,...,...,...,...,...,...,...
3672,B1,N,52,M,526,21.0,Physical,Government,NaN,NaN,2021-09-24
3678,C2,Y,23,F,219,7.0,Physical,Medium,NaN,2021-08-03,2021-08-03
3681,C1,N,59,F,703,403.0,Mental,Small,NaN,NaN,2021-07-05
3682,B1,Y,25,M,203,6.0,Physical,Small,NaN,2021-09-27,2021-07-13


In [21]:
predict_df.Gender = le1.fit_transform(predict_df.Gender)
predict_df.Physical_or_Mental_Injury = le2.fit_transform(predict_df.Physical_or_Mental_Injury)
predict_df.Employer_Type = le3.fit_transform(predict_df.Employer_Type)
predict_df.head(10)

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
70,AV,Y,69,1,406,259.0,1,4,NaN,2021-10-25,2021-07-22
181,B2,Y,63,0,533,164.0,1,4,NaN,2021-10-26,2021-06-18
1724,B2,N,41,0,542,213.0,1,2,NaN,NaN,2021-06-17
2061,B1,Y,44,0,527,100.0,1,1,NaN,2021-07-30,2021-06-09
2227,C1,N,53,1,542,117.0,1,4,NaN,NaN,2021-07-01
2327,C1,Y,57,1,799,140.0,1,2,NaN,2021-06-07,2021-06-25
2357,DH,Y,53,1,526,62.0,1,3,NaN,2021-09-27,2021-07-12
2367,B2,N,51,1,527,18.0,1,3,NaN,NaN,2021-06-07
2368,A1,Y,24,1,149,20.0,1,3,NaN,2021-07-25,2021-06-07
2369,DH,N,41,1,149,13.0,1,3,NaN,NaN,2021-06-07


In [22]:
predict = predict_df.drop(["Working", "Status26plus", "ARD", 'RTW_Date', "Team"], axis=1)

In [23]:
predict.head()

,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type
70,69,1,406,259.0,1,4
181,63,0,533,164.0,1,4
1724,41,0,542,213.0,1,2
2061,44,0,527,100.0,1,1
2227,53,1,542,117.0,1,4


In [24]:
predict_scaler = MinMaxScaler().fit(predict)
predict_test_scaled = predict_scaler.transform(predict)
predict_test_scaled

array([[0.8030303 , 1.        , 0.3386545 , 0.10389089, 1.        ,
        1.        ],
       [0.71212121, 0.        , 0.48346636, 0.0657842 , 1.        ,
        1.        ],
       [0.37878788, 0.        , 0.49372862, 0.08543923, 1.        ,
        0.5       ],
       ...,
       [0.65151515, 0.        , 0.67730901, 0.16165263, 0.        ,
        1.        ],
       [0.13636364, 1.        , 0.10718358, 0.00240674, 1.        ,
        1.        ],
       [0.36363636, 1.        , 0.33523375, 0.02406739, 1.        ,
        0.75      ]])

In [25]:
encoded_predictions = model.predict_classes(predict_test_scaled[:])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [26]:
predict_df = predict_df.drop(['Working'], axis=1)
predict_df['Working'] = prediction_labels
predict_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,1,406,259.0,1,4,NaN,2021-10-25,2021-07-22,N
181,B2,63,0,533,164.0,1,4,NaN,2021-10-26,2021-06-18,N
1724,B2,41,0,542,213.0,1,2,NaN,NaN,2021-06-17,Y
2061,B1,44,0,527,100.0,1,1,NaN,2021-07-30,2021-06-09,Y
2227,C1,53,1,542,117.0,1,4,NaN,NaN,2021-07-01,N
...,...,...,...,...,...,...,...,...,...,...,...
3672,B1,52,1,526,21.0,1,1,NaN,NaN,2021-09-24,Y
3678,C2,23,0,219,7.0,1,3,NaN,2021-08-03,2021-08-03,Y
3681,C1,59,0,703,403.0,0,4,NaN,NaN,2021-07-05,N
3682,B1,25,1,203,6.0,1,4,NaN,2021-09-27,2021-07-13,Y


In [27]:
predict_df.Gender = le1.inverse_transform(predict_df.Gender)
predict_df.Physical_or_Mental_Injury = le2.inverse_transform(predict_df.Physical_or_Mental_Injury)
predict_df.Employer_Type = le3.inverse_transform(predict_df.Employer_Type)
predict_df.head()

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22,N
181,B2,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18,N
1724,B2,41,F,542,213.0,Physical,Large,NaN,NaN,2021-06-17,Y
2061,B1,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09,Y
2227,C1,53,M,542,117.0,Physical,Small,NaN,NaN,2021-07-01,N


In [28]:
model_df.Gender = le1.inverse_transform(model_df.Gender)
model_df.Physical_or_Mental_Injury = le2.inverse_transform(model_df.Physical_or_Mental_Injury)
model_df.Employer_Type = le3.inverse_transform(model_df.Employer_Type)
model_df

C:\Users\Guest1\anaconda3\lib\site-packages\pandas\core\generic.py:5507: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
0,A2,Y,51,F,501,189.0,Physical,Medium,RTW,2021-02-25,2020-11-06
1,C2,Y,60,M,721,42.0,Physical,Small,RTW,2021-03-15,2020-11-05
2,C2,Y,29,F,459,156.0,Physical,Small,RTW,2021-12-02,2020-11-01
3,DH,N,19,M,224,4.0,Physical,Small,No RTW,2021-11-09,2020-11-02
4,C2,N,45,F,459,154.0,Physical,Medium,No RTW,NaN,2020-11-02
...,...,...,...,...,...,...,...,...,...,...,...
3675,B1,Y,55,M,721,5647.0,Physical,Medium,RTW,2021-06-14,2020-12-18
3676,B1,N,56,M,222,133.0,Physical,Medium,No RTW,2021-10-11,2021-04-08
3677,C2,Y,24,F,459,16.0,Physical,Medium,RTW,2021-07-28,2021-05-04
3679,AV,N,50,F,219,10.0,Physical,Small,No RTW,NaN,2021-02-26


In [29]:
predict_df.head()

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22,N
181,B2,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18,N
1724,B2,41,F,542,213.0,Physical,Large,NaN,NaN,2021-06-17,Y
2061,B1,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09,Y
2227,C1,53,M,542,117.0,Physical,Small,NaN,NaN,2021-07-01,N


In [30]:
working_df = modelling[modelling['Status26plus'].isna()]
working_df = working_df.dropna(subset=['RTW_Date'])
working_df

,Team,Working,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD
70,AV,Y,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22
181,B2,Y,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18
2061,B1,Y,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09
2327,C1,Y,57,M,799,140.0,Physical,Large,NaN,2021-06-07,2021-06-25
2357,DH,Y,53,M,526,62.0,Physical,Medium,NaN,2021-09-27,2021-07-12
...,...,...,...,...,...,...,...,...,...,...,...
3665,A1,Y,45,F,531,11.0,Physical,Large,NaN,2021-10-13,2021-09-30
3667,C1,Y,38,M,119,10.0,Physical,Small,NaN,2021-12-20,2021-09-30
3678,C2,Y,23,F,219,7.0,Physical,Medium,NaN,2021-08-03,2021-08-03
3682,B1,Y,25,M,203,6.0,Physical,Small,NaN,2021-09-27,2021-07-13


In [31]:
merged_df = pd.concat([predict_df, model_df])
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22,N
181,B2,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18,N
1724,B2,41,F,542,213.0,Physical,Large,NaN,NaN,2021-06-17,Y
2061,B1,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09,Y
2227,C1,53,M,542,117.0,Physical,Small,NaN,NaN,2021-07-01,N
...,...,...,...,...,...,...,...,...,...,...,...
3675,B1,55,M,721,5647.0,Physical,Medium,RTW,2021-06-14,2020-12-18,Y
3676,B1,56,M,222,133.0,Physical,Medium,No RTW,2021-10-11,2021-04-08,N
3677,C2,24,F,459,16.0,Physical,Medium,RTW,2021-07-28,2021-05-04,Y
3679,AV,50,F,219,10.0,Physical,Small,No RTW,NaN,2021-02-26,N


In [32]:
merged_df["ARD"] = merged_df["ARD"].apply(pd.to_datetime)
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,M,406,259.0,Physical,Small,NaN,2021-10-25,2021-07-22,N
181,B2,63,F,533,164.0,Physical,Small,NaN,2021-10-26,2021-06-18,N
1724,B2,41,F,542,213.0,Physical,Large,NaN,NaN,2021-06-17,Y
2061,B1,44,F,527,100.0,Physical,Government,NaN,2021-07-30,2021-06-09,Y
2227,C1,53,M,542,117.0,Physical,Small,NaN,NaN,2021-07-01,N
...,...,...,...,...,...,...,...,...,...,...,...
3675,B1,55,M,721,5647.0,Physical,Medium,RTW,2021-06-14,2020-12-18,Y
3676,B1,56,M,222,133.0,Physical,Medium,No RTW,2021-10-11,2021-04-08,N
3677,C2,24,F,459,16.0,Physical,Medium,RTW,2021-07-28,2021-05-04,Y
3679,AV,50,F,219,10.0,Physical,Small,No RTW,NaN,2021-02-26,N


In [33]:
merged_df["ARD"] = merged_df["ARD"].dt.strftime("%m/%y")
merged_df.dtypes

Team                          object
Age                            int64
Gender                        object
Injury_Code                    int64
Lodgement_Gap                float64
Physical_or_Mental_Injury     object
Employer_Type                 object
Status26plus                  object
RTW_Date                      object
ARD                           object
Working                       object
dtype: object

In [34]:
merged_df

,Team,Age,Gender,Injury_Code,Lodgement_Gap,Physical_or_Mental_Injury,Employer_Type,Status26plus,RTW_Date,ARD,Working
70,AV,69,M,406,259.0,Physical,Small,NaN,2021-10-25,07/21,N
181,B2,63,F,533,164.0,Physical,Small,NaN,2021-10-26,06/21,N
1724,B2,41,F,542,213.0,Physical,Large,NaN,NaN,06/21,Y
2061,B1,44,F,527,100.0,Physical,Government,NaN,2021-07-30,06/21,Y
2227,C1,53,M,542,117.0,Physical,Small,NaN,NaN,07/21,N
...,...,...,...,...,...,...,...,...,...,...,...
3675,B1,55,M,721,5647.0,Physical,Medium,RTW,2021-06-14,12/20,Y
3676,B1,56,M,222,133.0,Physical,Medium,No RTW,2021-10-11,04/21,N
3677,C2,24,F,459,16.0,Physical,Medium,RTW,2021-07-28,05/21,Y
3679,AV,50,F,219,10.0,Physical,Small,No RTW,NaN,02/21,N


In [35]:
merged_df_rtw = merged_df.loc[(merged_df['Working']=="Y")]

monthly_rtw = merged_df_rtw.groupby(["ARD"])
month_rtw_count = monthly_rtw['Working'].count()
monthly_rtw_df = pd.DataFrame({"RTW":month_rtw_count}).reset_index()

merged_df_nortw = merged_df.loc[(merged_df['Working']=="N")]

monthly_nortw = merged_df_nortw.groupby(["ARD"])
month_nortw_count = monthly_nortw['Working'].count()
monthly_nortw_df = pd.DataFrame({"No_RTW":month_nortw_count}).reset_index()


In [36]:
merged_monthly_df = monthly_rtw_df.merge(monthly_nortw_df, how="outer", on="ARD")
merged_monthly_df

,ARD,RTW,No_RTW
0,01/21,167,86
1,02/21,234,131
2,03/21,271,117
3,04/21,205,111
4,05/21,242,111
5,06/21,257,79
6,07/21,318,76
7,08/21,283,65
8,09/21,218,47
9,11/20,243,94


In [37]:
merged_monthly_df["rtw_pct"] = merged_monthly_df["RTW"] / (merged_monthly_df["RTW"] + merged_monthly_df["No_RTW"])
merged_monthly_df

,ARD,RTW,No_RTW,rtw_pct
0,01/21,167,86,0.660079
1,02/21,234,131,0.641096
2,03/21,271,117,0.698454
3,04/21,205,111,0.648734
4,05/21,242,111,0.685552
5,06/21,257,79,0.764881
6,07/21,318,76,0.807107
7,08/21,283,65,0.813218
8,09/21,218,47,0.822642
9,11/20,243,94,0.721068


In [38]:
merged_monthly_df.to_csv('../Resources/final.csv', index=False)